# CS88 Lecture 10 - Exceptions

## Exceptions

What happens when your program attempts to do something that just can't be done?

This should not be normal.  It should be rare!  Typically happens when your program encounters and *exceptional* situation

In [1]:
3/0

ZeroDivisionError: division by zero

In [2]:
str.lower(1)

TypeError: descriptor 'lower' requires a 'str' object but received a 'int'

In [2]:
""[3]

IndexError: string index out of range

In [3]:
3 % 0

ZeroDivisionError: integer division or modulo by zero

## Q: What should a function do?

## A: One thing well.

## Q: What should it do if it is passed arguments that don't make sense?

In [4]:
def divides(x, y):
    return y%x == 0

In [5]:
def get(data, selector):
    return data[selector]

In [6]:
divides(2,4)

True

In [7]:
divides(0,5)

ZeroDivisionError: integer division or modulo by zero

In [8]:
get([1,2,3],0)

1

In [9]:
get({'a': 34, 'cat':'9 lives'}, 'dog')

KeyError: 'dog'

In [10]:
get([1,2,3],[2])

TypeError: list indices must be integers or slices, not list

When an error is encountered the python interpreter *throws an exception*.  Here returns all the way to the top level and reports a stack trace of where the exception occured.

In [12]:
def divides(x, y):
    return y%x == 0
def divides24(x):
    return divides(x,24)
divides24(0)

ZeroDivisionError: integer division or modulo by zero

In [15]:
divides24(0)

ZeroDivisionError: integer division or modulo by zero

In [17]:
def mapply(f, s):
    return [f(x) for x in s]

In [18]:
mapply(divides24,[6,4,3,5])

[True, True, True, False]

In [19]:
mapply(divides24,[6, 4, 0, 3, 5])

ZeroDivisionError: integer division or modulo by zero

Many types of exceptions:

* `TypeError` -- A function was passed the wrong number/type of argument
* `NameError` -- A name wasn't found
* `KeyError` -- A key wasn't found in a dictionary
* `RuntimeError` -- Catch-all for troubles during interpretation

The flow of control stops at the exception and is 'thrown back'. Here the return (and the print) is not executed if an exception occurs on the divide.

In [19]:
def noisy_divides(x, y):
    result = (y % x == 0)
    if result:
        print("{0} divides {1}".format(x, y))
    else:
        print("{0} does not divide {1}".format(x, y))
    return result

In [20]:
noisy_divides(4,24)

4 divides 24


True

In [21]:
noisy_divides(0,24)

ZeroDivisionError: integer division or modulo by zero

In [29]:
def divides24(x):
    return noisy_divides(x,24)

In [30]:
divides24(0)

ZeroDivisionError: integer division or modulo by zero

## Assertions

Your functions should do all they can to avoid errors, they should handle them gracefully when they occur, and the should not trust that they are called with valid arguments -
*treat data as dirty till you've washed it*.

The most common form of throwing exceptions is with the `assert` statement.  Use it generously. Make sure that you code is working on something reasonable before it tries to do its stuff.  It serves as good documentation of the assumptions that your code makes.  And it avoids lots of very obscure bugs.

    asset <assertion expression>, <string for failed assertion>
    
Assert statements raise an exception of type `AssertionError`

In [33]:
def divides(x, y):
    assert x != 0, "Bad argument to divides - denominator should be non-zero"
    assert (type(x) == int and type(y) == int), "divides only takes integers"
    return y%x == 0

In [34]:
divides(0,3)

AssertionError: Bad argument to divides - denominator should be non-zero

In [35]:
divides(9, "lives")

AssertionError: divides only takes integers

In [4]:
def divides24(x):
    return divides(x,24)

In [5]:
mapply(divides24,[6,0,4,3,5])

NameError: name 'mapply' is not defined

## Handling errors

How can you continue in the presence of an error?  Is there a way to *handle the exception*?

The general form of this construct is

    try:
        <try suite>
    except <exception class> as <name>:
        <except suite>
    ... # continue here if <try suite> succeeds without exception

In [13]:
def safe_apply_fun(f,x):
    try:
        return f(x)   # normal execution, return the result
    except:           # error occured, f cannot return.  Transfer control back to here
        return "Invalid"   # value returned on exception

In [14]:
def divides(x, y):
    return y%x == 0
def divides24(x):
    return divides(x,24)
safe_apply_fun(divides24,0)

'Invalid'

In [29]:
def mapply(f, s):
    return [safe_apply_fun(f,x) for x in s]

In [16]:
mapply(divides24,[6,0,4,3,5])

[True, 'Invalid', True, True, False]

In [20]:
def rapply(f, s):
    if len(s) == 0:
        return []
    else:
        return [f(s[0])] + rapply(f, s[1:])

In [21]:
rapply(divides24, [6,4,3,5])

[True, True, True, False]

In [22]:
rapply(divides24, [6,4,3,0,5])

ZeroDivisionError: integer division or modulo by zero

In [23]:
def rapply(f, s):
    if len(s) == 0:
        return []
    else:
        return [safe_apply_fun(f, s[0])] + rapply(f, s[1:])

In [24]:
rapply(divides24, [6,4,3,0,5])

[True, True, True, 'Invalid', False]

In [25]:
def safe_apply_fun(f,x):
    try:
        return f(x)   # normal execution, return the result
    except Exception as e:  # exceptions are objects of class derived from base class Exception
        return e   # value returned on exception

In [26]:
safe_apply_fun(divides24,0)

ZeroDivisionError('integer division or modulo by zero')

In [30]:
res = mapply(divides24, [6,4,3,0,5])
res

[True,
 True,
 True,
 ZeroDivisionError('integer division or modulo by zero'),
 False]

In [31]:
res[3]

ZeroDivisionError('integer division or modulo by zero')

In [32]:
type(res[3])

ZeroDivisionError

## More on except

The general form of this construct is

    try:
        <try suite>
    except <exception class> as <name>:
        <except suite>
    ... # continue here if <try suite> succeeds without exception

Execution rule:
The `<try suite>` is executed first.
If during the course of executing the `<try suite>`
* an exception is raised that is not handled otherwise, and
* if the class of the exception inherits from `<exception class>`, then
* the `<except suite>` is executed, with `<name>` bound to the exception

Note:
* There can be more than one `except` clause for a `try`.
* They may specify a tuple of exception types.
* The first one that catches the exception receives control.
* If none do (or if there is no `try ... except`) control is thrown out of the function call.
* Each of the function calls on the stack may define exception handlers.  Control is transferred to nearest catching exception suite on the stack.

In [ ]:
def safe_apply_fun(f,x):
    try:
        return f(x)   # normal execution, return the result
    except AssertionError as e:
        return "Failed Assertion"
    except (TypeError, NameError):
        return "Bad function or arg type"

In [ ]:
safe_apply_fun(divides24, 0)

In [ ]:
safe_apply_fun("foo", 0)

In [ ]:
safe_apply_fun(divides25, 0)

In [ ]:
safe_apply_fun(lambda x: 24 % x == 0, 0)

## Raising your own exceptions

Exceptions are raised with a `raise` statement:

`raise <expression>`

`<expression>` must evaluate to a subclass of BaseException or an instance of one

Exceptions are constructed like any other object. E.g., `TypeError('Bad argument!')`

In [42]:
TypeError("ugly type")

TypeError('ugly type')

In [51]:
assert 1 < 2, "My Assertion"

In [39]:
def divides(x, y):
    assert x != 0, "Bad argument to divides - denominator should be non-zero"
    if (type(x) != int or type(y) != int):
        raise TypeError("divides only takes integers")
    return y%x == 0

In [40]:
divides("cat",9)

TypeError: divides only takes integers

In [41]:
safe_apply_fun(divides24, "cat")

TypeError('divides only takes integers')

## Exceptions are classes

They have constructors, selectors, methods, etc.

In [ ]:
# Exceptions are classes too
class NoiseyException(Exception):
    def __init__(self, stuff):
        print("Bad stuff happenned", stuff)

In [ ]:
def nostop(fun, x):
    try: 
        try:
            return fun(x)
        except:
            raise NoiseyException((fun, x))
    except:
        return None

In [ ]:
def reciprocal(x):
    return 1/x
nostop(reciprocal, 0)

In [ ]:
def zapper(fun, seq, selectors):
    return [nostop(fun, seq[x]) for x in selectors]

In [ ]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3])

In [ ]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3, 4])

In [ ]:
def zing(seq, i):
    try: 
        try:
            return seq[i]
        except:
            raise NoiseyException(("bad sequence index", i))
    except:
        return None

def zapper(fun, seq, selectors):
    return [nostop(fun, zing(seq, x)) for x in selectors]

In [ ]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3, 4])

In [ ]:
class NoiseyException(Exception):
    exceptions = []
    def __init__(self, stuff):
        print("Bad stuff happenned", stuff)
        NoiseyException.exceptions.append(stuff)

In [ ]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3, 4])

In [ ]:
NoiseyException.exceptions

# Part 2: Iterators:

# CS88 Lecture 12 - Iterators and Iterables

A *sequence* is something that you can:
* index into
* get the length of.

In [ ]:
[1,2,3]

In [3]:
s = [1,2,3,4]
s

[1, 2, 3, 4]

In [4]:
len(s)

4

In [5]:
s[1]

2

In [6]:
"Hello CS88 World"[4]

'o'

What are some examples of types sequences?
- list
- tuple
- string

Note: a `dict` is *not* a sequence


## Iterable - an object you can iterate over

* *iterable*: An object capable of yielding its members one at a time.
* *iterator*: An object representing a stream of data.

We have worked with many iterables as if they were sequences

In [7]:
# list comprehension over an iterable
[x*x for x in s]

[1, 4, 9, 16]

In [8]:
# for loop over one too
for x in s:
    print(x)

1
2
3
4


In [ ]:
# iteration, but not an iteration over a sequence
while s:
    print(s)
    s = s[1:]

In [9]:
# Please, please, please do NOT do this
#   - there is no recurrence 
#   - we are just mapping each element to a value

def squaresie(s):
    res = []
    for i in range(len(s)):
        res.append(s[i]*s[i])
    return res

# mutation for no reason - should be a last resort
# indexing when we only want the elements - we don't need the position

In [11]:
squaresie(s)

[1, 4, 9, 16]

In [12]:
range(10)

range(0, 10)

In [ ]:
def squares(s):
    return [x*x for x in s]

In [ ]:
s = [1,2,3,4]
squares(s)

In [ ]:
map(lambda x: x*x, s)

In [ ]:
def anyone(x):
    anyx = False
    for e in x:
        anyx = anyx or bool(e)
    return anyx

In [ ]:
anyone([0, False, [], ''])

In [ ]:
help(any)

In [ ]:
def anyone(x):
    for e in x:
        if e:
            return True
    return False

In [ ]:
anyone([0, False, [], ''])

# Functions that return iterables

- `map`
- `range`
- `zip`

These objects are not sequences.

If we want to see all of the elements at once, we need to explicitly call
`list()` or `tuple()` on them

In [ ]:
map(lambda x: x*x, [1,2,3])

In [14]:
range(5)

range(0, 5)

In [15]:
map(lambda x: x*x, range(5))

In [16]:
[x for x in map(lambda x: x*x, range(5))]

[0, 1, 4, 9, 16]

In [18]:
list(map(lambda x: x*x, range(5)))

[0, 1, 4, 9, 16]

In [19]:
for x in map(lambda x: x*x, range(5)):
  print(x)

0
1
4
9
16


In [20]:
map(lambda x: x*x, range(5))[3]

TypeError: 'map' object is not subscriptable

In [ ]:
zip([1,2,3], ['a', 'b'])

In [ ]:
list(zip([1,2,3], ['a', 'b']))

## Motivating question

How can we define objects that behave like sequences without explicitly creating the sequence?
* Generate each of the objects as we access them


"A prime number (or a prime) is a natural number greater than 1 that cannot be formed by multiplying two smaller natural numbers."

In [ ]:
def prime(n):
    return n > 1 and not any([n % i == 0 for i in range(2, n)])

In [ ]:
list(zip(range(10), map(prime, range(10))))

Why might we want to do this?  
- when the data is really BIG
- maybe infinite!

We need a concept of *lazy evaluation* - only compute what you need

In [ ]:
range(100000000000)

In [ ]:
x = map(prime, range(100000000000))
x

In [ ]:
for i,p in zip(range(10), x):
    print(i,p)

In [ ]:
[(i,p) for i,p in zip(range(10), x)]

In [ ]:
def first(n, x):
    return [e for i, e in zip(range(n), x)]

In [ ]:
first(10, x)

# Extra Stuff (Generators):

## Generators: turning iteration into an interable

- *Generator* functions use iteration (for loops, while loops) and the `yield` keyword
- Generator functions have no return statement, but they don’t return None
- They implicitly return a generator object
- Generator objects are just iterators

In [ ]:
def (n):
    for i in rangesquaresp(n):
        print (i*i)

In [ ]:
squaresp(5)

In [ ]:
def squares(n):
    for i in range(n):
        yield (i*i)

In [ ]:
squares(5)

In [ ]:
list(squares(5))

In [ ]:
from math import sqrt

In [ ]:
map(sqrt, squares(6))

In [ ]:
[i for i in map(sqrt, squares(6))]

In [ ]:
squares(100)[6]

In [ ]:
def isprimes(n):
    for i in range(n):
        yield (prime(i))

In [ ]:
first(5, isprimes(10000000))

In [ ]:
def primes():
    i = 2
    while True:
        if prime(i):
            yield(i)
        i += 1

In [ ]:
primes()

In [ ]:
first(10, primes())

In [ ]:
def squares2(n):
    i = 0
    while i < n:
        yield(i*i)
        i += 1

In [ ]:
[i for i in map(sqrt, squares2(6))]

In [ ]:
# an infinite object
def all_squares():
    i = 0
    while True:
        yield(i*i)
        i += 1

In [ ]:
all_squares()

In [ ]:
[(i,x) for i,x in zip(range(10),all_squares())]

## Nested iteration

In [ ]:
def all_pairs(x):
    for item1 in x:
        for item2 in x:
            yield(item1, item2)

In [ ]:
all_pairs(['apple', 'banana', 'orange'])

In [ ]:
list(all_pairs(['apple', 'banana', 'orange']))

In [ ]:
# nested iteration is available in list comprehensions too
[(i, j) for i in range(4) for j in range(3) ]

# Sequences and Iterables


## Next element in generator iterable

Iterables work because they have some "magic methods" on them.  We saw magic methods when we learned about classes, e.g., `__init__`, `__repr__` and `__str__`.  

The first one we see for iterables is `__next__`

In [ ]:
x = all_squares()

In [ ]:
x

In [ ]:
help(next)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
x

In [ ]:
x.__next__

## iter - transforming a sequence into an interator

Builtin function iter takes an iterable object, e.g., a sequence, and returns an iterator

In [ ]:
help(iter)

In [ ]:
x = iter([1,2,3])

In [ ]:
x

In [ ]:
next(x)

In [ ]:
x.__next__()

In [ ]:
x[3]

In [ ]:
[x for x in iter([1,2,3])]

In [ ]:
y = all_squares()

In [ ]:
next(y)

In [ ]:
next(y)

In [ ]:
iter(y)

In [ ]:
next(y)

In [ ]:
s = "abc"
xs = iter(s)

In [ ]:
next(xs)

In [ ]:
next(xs)

In [ ]:
next(xs)

In [ ]:
next(xs)

In [ ]:
sq = all_squares()

In [ ]:
sq

In [ ]:
next(sq)

In [ ]:
sq.__next__()

# Iterators

In order to be *iterable*, a class must implement the `iter` protocol 

The iterator objects themselves are required to support the following two methods, which together form the iterator protocol:

* `__iter__()` : Return the iterator object itself. This is required to allow both containers and iterators to be used with the for and in statements.
- This method returns an iterator object
- Iterator can be self

* `__next__()` : Return the next item from the container. If there are no further items, raise the `StopIteration` exception.

Classes get to define how they are iterated over by defining these methods

In [ ]:
class myrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __iter__(self):
        return self

    def __next__(self):
        if self.i < self.n:
            i = self.i
            self.i += 1
            return i
        else:
            raise StopIteration()

In [ ]:
myrange(5)

In [ ]:
[x for x in myrange(5)]

### `__next__(self)`

Accessed via the next method
* Returns the next element in the iteration
    - Must keep track of where it is in the sequence
* Once there are no more items left in the sequence, raise an exception:
    - raise StopIteration

In [ ]:
x = myrange(2)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

### pro·to·col:

* the official procedure or system of rules governing affairs of state or diplomatic occasions.

* COMPUTING:
a set of rules governing the exchange or transmission of data between devices.

* a formal or official record of scientific experimental observations.
a procedure for carrying out a scientific experiment or a course of medical treatment.

## Getitem protocol

Another way an object can behave like a sequence is *indexing*: Using square brackets “[ ]” to access specific items in an object.

* Defined by special method: __getitem__(self, i) 
     - Method returns the item at a given index
* As the designers of the class, get to decide what an index represents:
    - Sequences: The item at a position in the sequence
    - Dictionaries: The value associated with a given key
    - Arrays: Index is a tuple representing the coordinate of the item

A class that does not support __iter__ but with a __getitem__ method that raises an IndexError exception if the index gets to large is also iterable.

In [ ]:
class myrange2:
    def __init__(self, n):
        self.n = n
        
    def __getitem__(self, i):
        if i >= 0 and i < self.n:
            return i
        else:
            raise IndexError
    
    def __len__(self):
        return self.n

In [ ]:
x = myrange2(4)

In [ ]:
len(x)

In [ ]:
[x for x in myrange2(3)]

In [ ]:
x[2]

## Determining if an object is iterable

This is more general than checking for any list of particular type, e.g., list, tuple, string...

In [1]:
from collections.abc import Iterable
from collections.abc import Sequence


isinstance([1,2,3], Iterable)

True

In [ ]:
isinstance((1,2,3), Iterable)

In [ ]:
isinstance({'a':1, 'b':2}, Iterable)

In [ ]:
isinstance('s', Iterable)

In [ ]:
isinstance('s'[0], Iterable)

In [ ]:
myrange

In [ ]:
myrange.__iter__

In [ ]:
isinstance(myrange(4), Iterable)

In [ ]:
isinstance(myrange2(4), Iterable)

In [ ]:
isinstance(all_squares(), Iterable)

# Extra for Experience:

## Examples

In [2]:
# Get input from the user as a string
input()

test


'test'

In [ ]:
def input_stream():
    """Stream input till empty rtn"""
    user = True
    while user:
        user = input()
        yield(user)

In [ ]:
istream = input_stream()

In [ ]:
istream

In [ ]:
list(istream)

### Using iterators for lazy evaluation

Let's start with a recursive function to flatten a tree structure

In [ ]:
def concat(s, lvl=1):
    """Concatenate a sequence of sequences."""
#    print('  s:', '-'*lvl, s)
    conc = []
    for e in s:
#        print("Cat:", '-'*lvl, e)
        conc = conc + e
    return conc

def flatten(tree, lvl=1):
    if isinstance(tree, str) or not isinstance(tree, Iterable): 
        return [tree]  # a leaf node
    else:
        return concat([flatten(branch, lvl+1) for branch in tree], lvl)

In [ ]:
concat([[1,2,3],[4,5,7], [6]])

In [ ]:
flatten([1, 3, ['a','foo'], range(9,13)])

In [ ]:
def iconcat(s, lvl=1):
    """Generate a concatenation of a sequence of sequences."""
#    print(" s:", "-"*lvl, s)
    for e in s:
        for x in e:
#            print("IC:", "-"*lvl, x)
            yield(x)

def iflatten(tree, lvl=1):
    if isinstance(tree, str) or not isinstance(tree, Iterable):   
        return [tree]  # a leaf node
    else:
        return iconcat([iflatten(branch, lvl+1) for branch in tree], lvl)

In [ ]:
iconcat([[1,2,3], [4,5], [6]])

In [ ]:
list(iconcat([[1,2,3], [4,5], [6]]))

In [ ]:
list(iflatten([1,3,['a','foo'],range(9,13)]))

In [ ]:
list(iflatten([[1,2],3,[[4]]]))

In [ ]:
def equal_fringe(tree1, tree2):
    for i1, i2 in zip(iflatten(tree1), iflatten(tree2)):
        if not i1 == i2:
            return False
    return True

In [ ]:
equal_fringe([1, [2, [3,4]]], [[1,2], 3, [4]])

In [ ]:
equal_fringe([1,2,[3,4]], [[7,4],3,[4]])